In [102]:
import logging
from pprint import pprint

from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI
from vertexai.generative_models import GenerativeModel

In [2]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

# Vetex AI

In [3]:
# model = GenerativeModel("gemini-1.0-pro")
model = GenerativeModel("gemini-1.5-pro-preview-0409")

In [4]:
responses = model.generate_content("Why is the sky blue?", stream=True)

for response in responses:
    print(response.text, end="")

The sky appears blue due to a phenomenon called Rayleigh scattering. Sunlight reaches Earth's atmosphere and is scattered in all directions by the gases and particles in the air. Blue light is scattered more than other colors because it travels as shorter, smaller waves. This is why we see a blue sky most of the time. 


# Langchain

## OpenAI

In [103]:
openai_llm = ChatOpenAI(model="gpt-4-turbo-preview")

In [104]:
openai_llm.invoke(input="Why is the sky blue?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="The sky appears blue to the human eye because of a phenomenon called Rayleigh scattering. This scattering occurs when sunlight enters the Earth's atmosphere and interacts with air molecules, water droplets, and tiny particles. Sunlight, or white light, is made up of a spectrum of colors that can be seen in a rainbow. Each color has a different wavelength, with blue light having shorter, smaller waves.\n\nWhen sunlight encounters the molecules and particles in the Earth's atmosphere, the shorter wavelengths of light (blue and violet) are scattered in all directions by these molecules and particles more than the other colors in the spectrum. While violet light is also scattered, our eyes are more sensitive to blue light, and the upper atmosphere absorbs violet light, making the sky appear predominantly blue to us.\n\nThe scattering of shorter wavelengths of light more than the longer wavelengths (like red and orange) is due to the sizes of the atmospheric particles bei

## Gemini

In [105]:
# llm = ChatVertexAI(model="gemini-1.0-pro")
# llm = ChatVertexAI(model="gemini-1.0-pro-001")
# llm = ChatVertexAI(model="gemini-1.0-pro-002")
llm = ChatVertexAI(model="gemini-1.5-pro-preview-0409")

In [86]:
response = llm.invoke(input="Why is the sky blue?")
response

AIMessage(content="The sky appears blue due to a phenomenon called Rayleigh scattering. Sunlight reaches Earth's atmosphere and is scattered in all directions by the gases and particles in the air. Blue light is scattered more than other colors because it travels as shorter, smaller waves. This is why we see a blue sky most of the time. \n", response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'citation_metadata': {'citations': [{'start_index': 69, 'end_index': 213, 'uri': 'https://wordpress.digital4design.com/category/human-eye/', 'title': '', 'license_': ''}, {'start_index': 190, 'end_inde

In [87]:
pprint(response.dict())
print(len(response.content))

{'additional_kwargs': {},
 'content': 'The sky appears blue due to a phenomenon called Rayleigh '
            "scattering. Sunlight reaches Earth's atmosphere and is scattered "
            'in all directions by the gases and particles in the air. Blue '
            'light is scattered more than other colors because it travels as '
            'shorter, smaller waves. This is why we see a blue sky most of the '
            'time. \n',
 'example': False,
 'id': 'run-7df1b05e-0ac7-4262-a124-b3cf2dab25f6-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'citation_metadata': {'citations': [{'end_index': 213,
                                                            'license_': '',
                                                            'start_index': 69,
                                                            'title': '',
                                                            'uri': 'https://wordpress.digital4design.com/category/human-eye/'},
             

# Agent

In [106]:
tools = [TavilySearchResults(max_results=1)]

In [107]:
llm_with_tools = llm.bind_tools(tools)

In [90]:
# query = "What is LangChain?"
query = "Why is the sky blue?"

In [91]:
llm_with_tools.invoke(query).tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'Why is the sky blue?'},
  'id': 'cf581cb6-fa51-4ec9-ad8d-a38648d7d867'}]

In [108]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
            "You are a helpful assistant. You only use the tavily_search_results tool for recent information that you don't know.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## OpenAI

In [118]:
openai_agent = create_tool_calling_agent(openai_llm, tools, prompt)

In [119]:
openai_agent_executor = AgentExecutor(agent=openai_agent, tools=tools, verbose=True)

In [128]:
# openai_agent_executor.invoke({"input": "what is LangChain?"})
# openai_agent_executor.invoke({"input": "Why is the sky blue?"})
openai_agent_executor.invoke({"input": "How to wake up early?"})
# openai_agent_executor.invoke({"input": "What is the weather today in HCMC?"})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Waking up early can be a challenging habit to develop, but it's definitely achievable with the right strategies. Here are some tips to help you wake up early and make the most of your morning:

1. **Gradually Adjust Your Schedule**: Start by setting your alarm just 15 minutes earlier than usual and gradually adjust it earlier in small increments until you reach your desired wake-up time.

2. **Create a Sleep Schedule**: Go to bed and wake up at the same time every day, even on weekends. Consistency is key to adjusting your body's internal clock.

3. **Improve Your Sleep Environment**: Make sure your bedroom is conducive to sleep. It should be quiet, dark, and at a comfortable temperature. Consider using blackout curtains, earplugs, or white noise machines if necessary.

4. **Limit Exposure to Screens**: The blue light emitted by phones, tablets, and computers can interfere with your ability to fall asleep. Try to avoid screens for at least an hour before bed.

5. **Develop a Bedtime Ro

{'input': 'How to wake up early?',
 'output': "Waking up early can be a challenging habit to develop, but it's definitely achievable with the right strategies. Here are some tips to help you wake up early and make the most of your morning:\n\n1. **Gradually Adjust Your Schedule**: Start by setting your alarm just 15 minutes earlier than usual and gradually adjust it earlier in small increments until you reach your desired wake-up time.\n\n2. **Create a Sleep Schedule**: Go to bed and wake up at the same time every day, even on weekends. Consistency is key to adjusting your body's internal clock.\n\n3. **Improve Your Sleep Environment**: Make sure your bedroom is conducive to sleep. It should be quiet, dark, and at a comfortable temperature. Consider using blackout curtains, earplugs, or white noise machines if necessary.\n\n4. **Limit Exposure to Screens**: The blue light emitted by phones, tablets, and computers can interfere with your ability to fall asleep. Try to avoid screens for 

## Gemini

In [121]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [122]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [127]:
# agent_executor.invoke({"input": "what is LangChain?"})
# agent_executor.invoke({"input": "Why is the sky blue?"})
agent_executor.invoke({"input": "How to wake up early?"})
# agent_executor.invoke({"input": "What is the weather today in HCMC?"})



> Entering new AgentExecutor chain...
## Tips to Wake Up Early

Waking up early can be challenging, but it's definitely achievable with the right approach. Here are some tips to help you become an early riser:

**1. Gradual Adjustment:**

* **Start slow:** Don't jump from waking up at 9 am to 5 am overnight. Gradually shift your wake-up time by 15-30 minutes earlier every few days. This allows your body to adjust gently. 
* **Adjust bedtime accordingly:** Ensure you're getting enough sleep (7-8 hours for most adults). Calculate your ideal bedtime based on your desired wake-up time and stick to it.

**2. Optimize Your Sleep Environment:**

* **Make it dark:** Use blackout curtains or a sleep mask to block out light, which can interfere with melatonin production. 
* **Keep it quiet:** Minimize noise disruptions with earplugs or a white noise machine.
* **Cool it down:** A slightly cool room temperature (around 65°F or 18°C) promotes better sleep.
* **Invest in a comfortable bed and pil

{'input': 'How to wake up early?',
 'output': "## Tips to Wake Up Early\n\nWaking up early can be challenging, but it's definitely achievable with the right approach. Here are some tips to help you become an early riser:\n\n**1. Gradual Adjustment:**\n\n* **Start slow:** Don't jump from waking up at 9 am to 5 am overnight. Gradually shift your wake-up time by 15-30 minutes earlier every few days. This allows your body to adjust gently. \n* **Adjust bedtime accordingly:** Ensure you're getting enough sleep (7-8 hours for most adults). Calculate your ideal bedtime based on your desired wake-up time and stick to it.\n\n**2. Optimize Your Sleep Environment:**\n\n* **Make it dark:** Use blackout curtains or a sleep mask to block out light, which can interfere with melatonin production. \n* **Keep it quiet:** Minimize noise disruptions with earplugs or a white noise machine.\n* **Cool it down:** A slightly cool room temperature (around 65°F or 18°C) promotes better sleep.\n* **Invest in a co